# Imports

In [1]:
import pandas as pd
import numpy as np
import pymysql
import mysql.connector
from datetime import datetime

# Get processing date

In [2]:
processing_date = datetime.now()

# Load input file

Quick analysis on data shows that:
1. Column "has_mobile_app_tracking" is duplicated as "has_mobile_app_tracking.1"
2. Boolean values have to be replaced by strings 'TRUE' or 'FALSE'
3. nan values have to be replaced by None so they can be interpreted as NULL by MySQL

In [3]:
raw_data = pd.read_csv('DataChallenge.csv')

# Field has_mobile_app_tracking.1 is a duplication
raw_data.drop(columns=['has_mobile_app_tracking.1'], inplace=True)

# Converting bool fields to strings
raw_data['vip_carrier'].replace({True: 'TRUE', False: 'FALSE'}, inplace=True)
raw_data['has_mobile_app_tracking'].replace({True: 'TRUE', False: 'FALSE'}, inplace=True)
raw_data['has_macropoint_tracking'].replace({True: 'TRUE', False: 'FALSE'}, inplace=True)
raw_data['has_edi_tracking'].replace({True: 'TRUE', False: 'FALSE'}, inplace=True)
raw_data['contracted_load'].replace({True: 'TRUE', False: 'FALSE'}, inplace=True)
raw_data['load_booked_autonomously'].replace({True: 'TRUE', False: 'FALSE'}, inplace=True)
raw_data['load_sourced_autonomously'].replace({True: 'TRUE', False: 'FALSE'}, inplace=True)
raw_data['load_was_cancelled'].replace({True: 'TRUE', False: 'FALSE'}, inplace=True)

# Converting NumPy nan into None so it can be interpreted as NULL by MySQL
raw_data = raw_data.replace({np.nan: None})

# Add processing date to dataframe
raw_data['processing_date'] = processing_date

raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5361 entries, 0 to 5360
Data columns (total 31 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   loadsmart_id                 5361 non-null   int64         
 1   lane                         5361 non-null   object        
 2   quote_date                   5361 non-null   object        
 3   book_date                    5361 non-null   object        
 4   source_date                  4863 non-null   object        
 5   pickup_date                  5361 non-null   object        
 6   delivery_date                5361 non-null   object        
 7   book_price                   5361 non-null   object        
 8   source_price                 5361 non-null   object        
 9   pnl                          5361 non-null   object        
 10  mileage                      5361 non-null   object        
 11  equipment_type               5361 non-null 

Dimensions:
- LOCATION: city, state
- DATE: 
- TIME:
- SOURCING: sourcing_channel
- EQUIPMENT: equipment_type 
- CARRIER: carrier_name, carrier_rating, vip_carrier, carrier_dropped_us_count
- SHIPPER: shipper_name
- TRACKING: has_mobile_app_tracking, has_macropoint_tracking, has_edi_tracking

# Connect to MySQL

In [4]:
db = mysql.connector.connect(host='localhost', 
                             user='{your_user_name}', 
                             password='{your_password}', 
                             database='LS')

# Insert input file into STG_LS_RAW stagging table

In [5]:
cursor = db.cursor()

cursor.execute("TRUNCATE TABLE STG_LS_RAW")

sql = """
INSERT INTO STG_LS_RAW
(
loadsmart_id,  
lane, 
quote_date, 
book_date, 
source_date, 
pickup_date, 
delivery_date, 
book_price, 
source_price, 
pnl, 
mileage, 
equipment_type, 
carrier_rating, 
sourcing_channel, 
vip_carrier, 
carrier_dropped_us_count, 
carrier_name, 
shipper_name, 
carrier_on_time_to_pickup, 
carrier_on_time_to_delivery, 
carrier_on_time_overall, 
pickup_appointment_time, 
delivery_appointment_time, 
has_mobile_app_tracking, 
has_macropoint_tracking, 
has_edi_tracking, 
contracted_load, 
load_booked_autonomously,
load_sourced_autonomously, 
load_was_cancelled,
processing_date
)
VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)                               
"""

cursor.executemany(sql, raw_data.apply(tuple, axis=1).tolist())
db.commit()

# Insert records with important values/keys missing into STG_LS_BAD table

In [6]:
cursor = db.cursor()

sql = """
INSERT INTO STG_LS_BAD
SELECT loadsmart_id,  
       lane, 
       quote_date, 
       book_date, 
       source_date, 
       pickup_date, 
       delivery_date, 
       book_price, 
       source_price, 
       pnl, 
       mileage, 
       equipment_type, 
       carrier_rating, 
       sourcing_channel, 
       vip_carrier, 
       carrier_dropped_us_count, 
       carrier_name, 
       shipper_name, 
       carrier_on_time_to_pickup, 
       carrier_on_time_to_delivery, 
       carrier_on_time_overall, 
       pickup_appointment_time, 
       delivery_appointment_time, 
       has_mobile_app_tracking, 
       has_macropoint_tracking, 
       has_edi_tracking, 
       contracted_load, 
       load_booked_autonomously,
       load_sourced_autonomously, 
       load_was_cancelled,
       processing_date
FROM   STG_LS_RAW
WHERE  lane IS NULL
OR     book_date IS NULL
OR     pickup_date IS NULL
OR     delivery_date IS NULL
OR     book_price IS NULL
OR     source_price IS NULL
OR     mileage IS NULL
OR     carrier_name IS NULL
OR     shipper_name IS NULL   
""";

cursor.execute(sql)
db.commit()

# Remove duplicated records + normalizations + transformations + insert into STG_LS stagging table

In [7]:
cursor = db.cursor()

sql = "TRUNCATE TABLE STG_LS"

cursor.execute(sql)

sql = """
INSERT INTO STG_LS
WITH ALL_RECORDS AS 
(
SELECT loadsmart_id,  
       lane, 
       quote_date, 
       book_date, 
       source_date, 
       pickup_date, 
       delivery_date, 
       book_price, 
       source_price, 
       pnl, 
       mileage, 
       equipment_type, 
       carrier_rating, 
       sourcing_channel, 
       vip_carrier, 
       carrier_dropped_us_count, 
       carrier_name, 
       shipper_name, 
       carrier_on_time_to_pickup, 
       carrier_on_time_to_delivery, 
       carrier_on_time_overall, 
       pickup_appointment_time, 
       delivery_appointment_time, 
       has_mobile_app_tracking, 
       has_macropoint_tracking, 
       has_edi_tracking, 
       contracted_load, 
       load_booked_autonomously,
       load_sourced_autonomously, 
       load_was_cancelled,
       processing_date,
       ROW_NUMBER() OVER(PARTITION BY loadsmart_id, lane, quote_date, book_date, source_date, pickup_date, delivery_date, book_price, 
                                      source_price, pnl, mileage, equipment_type, carrier_rating, sourcing_channel, vip_carrier, 
                                      carrier_dropped_us_count, carrier_name, shipper_name, carrier_on_time_to_pickup, carrier_on_time_to_delivery,
                                      carrier_on_time_overall, pickup_appointment_time, delivery_appointment_time, has_mobile_app_tracking, 
                                      has_macropoint_tracking, has_edi_tracking, contracted_load, load_booked_autonomously,
                                      load_sourced_autonomously, load_was_cancelled) record_order
FROM   STG_LS_RAW
WHERE  lane IS NOT NULL
AND    book_date IS NOT NULL
AND    pickup_date IS NOT NULL
AND    delivery_date IS NOT NULL
AND    book_price IS NOT NULL
AND    source_price IS NOT NULL
AND    mileage IS NOT NULL
AND    carrier_name IS NOT NULL
AND    shipper_name IS NOT NULL 
)
SELECT V2.loadsmart_id, 
       L.location_id lane_origin_id,
       V2.lane_origin_city, 
       V2.lane_origin_state, 
       L2.location_id lane_destination_id,
       V2.lane_destination_city, 
       V2.lane_destination_state, 
       D.date_id quote_date_id,
       V2.quote_date,
       T.time_id quote_time_id,
       V2.quote_time,
       D2.date_id book_date_id,
       V2.book_date,
       T2.time_id book_time_id,
       V2.book_time,
       D3.date_id source_date_id,
       V2.source_date,
       T3.time_id source_time_id,
       V2.source_time,
       D4.date_id pickup_date_id,
       V2.pickup_date,
       T4.time_id pickup_time_id,
       V2.pickup_time,
       D5.date_id delivery_date_id,
       V2.delivery_date,
       T5.time_id delivery_time_id,
       V2.delivery_time,
       V2.book_price, 
       V2.source_price, 
       V2.pnl, 
       V2.mileage, 
       E.equipment_id,
       V2.equipment_type, 
       S.shipper_id,
       V2.shipper_name, 
       C.carrier_id,
       V2.carrier_name, 
       V2.carrier_rating, 
       V2.vip_carrier, 
       V2.carrier_dropped_us_count, 
       SO.sourcing_id,
       V2.sourcing_channel, 
       V2.carrier_on_time_to_pickup,
       V2.carrier_on_time_to_delivery,
       V2.carrier_on_time_overall,  
       D6.date_id pickup_appointment_date_id,
       V2.pickup_appointment_date,
       T6.time_id pickup_appointment_time_id,
       V2.pickup_appointment_time,
       D7.date_id delivery_appointment_date_id,
       V2.delivery_appointment_date,
       T7.time_id delivery_appointment_time_id,
       V2.delivery_appointment_time,
       TR.tracking_id,  
       V2.has_mobile_app_tracking,                         
       V2.has_macropoint_tracking,                         
       V2.has_edi_tracking,
       LO.load_id,
       V2.contracted_load, 
       V2.load_booked_autonomously, 
       V2.load_sourced_autonomously, 
       V2.load_was_cancelled,
       V2.processing_date
FROM   (SELECT V.loadsmart_id, 
               -- Lane split into CITY and STATE
               UPPER(TRIM(SUBSTRING(V.lane_origin, 1, INSTR(V.lane_origin, ',') - 1))) lane_origin_city, 
               UPPER(TRIM(SUBSTRING(V.lane_origin, INSTR(V.lane_origin, ',') + 1))) lane_origin_state, 
               UPPER(TRIM(SUBSTRING(V.lane_destination, 1, INSTR(V.lane_destination, ',') - 1))) lane_destination_city, 
               UPPER(TRIM(SUBSTRING(V.lane_destination, INSTR(V.lane_destination, ',') + 1))) lane_destination_state, 
               V.quote_date,
               V.quote_time,
               V.book_date,
               V.book_time,
               V.source_date,
               V.source_time,
               V.pickup_date,
               V.pickup_time,
               V.delivery_date,
               V.delivery_time,
               V.book_price, 
               V.source_price, 
               V.pnl, 
               V.mileage, 
               V.equipment_type, 
               V.shipper_name, 
               V.carrier_name, 
               V.carrier_rating, 
               V.vip_carrier, 
               V.carrier_dropped_us_count, 
               V.sourcing_channel, 
               V.carrier_on_time_to_pickup,
               V.carrier_on_time_to_delivery,
               V.carrier_on_time_overall,  
               V.pickup_appointment_date,
               V.pickup_appointment_time,
               V.delivery_appointment_date,
               V.delivery_appointment_time,
               V.has_mobile_app_tracking,                         
               V.has_macropoint_tracking,                         
               V.has_edi_tracking,
               V.contracted_load, 
               V.load_booked_autonomously, 
               V.load_sourced_autonomously, 
               V.load_was_cancelled,
               V.processing_date
        FROM   (SELECT loadsmart_id, 
                       -- Lane split into ORIGIN and DESTINATION
                       UPPER(TRIM(SUBSTRING(lane, 1, INSTR(lane, '->') - 2))) lane_origin, 
                       UPPER(TRIM(SUBSTRING(lane, INSTR(lane, '->') + 2))) lane_destination, 
                       -- String converted to Date
                       STR_TO_DATE(TRIM(SUBSTRING(quote_date, 1, INSTR(quote_date, ' ') - 1)), '%m/%d/%Y') quote_date,
                       STR_TO_DATE(TRIM(SUBSTRING(quote_date, INSTR(quote_date, ' ') + 1)), '%H:%i') quote_time,
                       STR_TO_DATE(TRIM(SUBSTRING(book_date, 1, INSTR(book_date, ' ') - 1)), '%m/%d/%Y') book_date,
                       STR_TO_DATE(TRIM(SUBSTRING(book_date, INSTR(book_date, ' ') + 1)), '%H:%i') book_time,
                       STR_TO_DATE(TRIM(SUBSTRING(source_date, 1, INSTR(source_date, ' ') - 1)), '%m/%d/%Y') source_date,
                       STR_TO_DATE(TRIM(SUBSTRING(source_date, INSTR(source_date, ' ') + 1)), '%H:%i') source_time,
                       STR_TO_DATE(TRIM(SUBSTRING(pickup_date, 1, INSTR(pickup_date, ' ') - 1)), '%m/%d/%Y') pickup_date,
                       STR_TO_DATE(TRIM(SUBSTRING(pickup_date, INSTR(pickup_date, ' ') + 1)), '%H:%i') pickup_time,
                       STR_TO_DATE(TRIM(SUBSTRING(delivery_date, 1, INSTR(delivery_date, ' ') - 1)), '%m/%d/%Y') delivery_date,
                       STR_TO_DATE(TRIM(SUBSTRING(delivery_date, INSTR(delivery_date, ' ') + 1)), '%H:%i') delivery_time,
                       book_price, 
                       source_price, 
                       pnl, 
                       mileage, 
                       equipment_type, 
                       UPPER(TRIM(shipper_name)) shipper_name, 
                       UPPER(TRIM(carrier_name)) carrier_name, 
                       carrier_rating, 
                       -- 'TRUE'/'FALSE' converted to number
                       CASE WHEN UPPER(TRIM(vip_carrier)) = 'TRUE' THEN 1
                            WHEN UPPER(TRIM(vip_carrier)) = 'FALSE' THEN 0
                            ELSE NULL
                       END vip_carrier, 
                       carrier_dropped_us_count, 
                       UPPER(TRIM(sourcing_channel)) sourcing_channel, 
                       -- 'TRUE'/'FALSE' converted to number
                       CASE WHEN UPPER(TRIM(carrier_on_time_to_pickup)) = 'TRUE' THEN 1
                            WHEN UPPER(TRIM(carrier_on_time_to_pickup)) = 'FALSE' THEN 0
                            ELSE NULL
                       END carrier_on_time_to_pickup,
                       CASE WHEN UPPER(TRIM(carrier_on_time_to_delivery)) = 'TRUE' THEN 1
                            WHEN UPPER(TRIM(carrier_on_time_to_delivery)) = 'FALSE' THEN 0
                            ELSE NULL
                       END carrier_on_time_to_delivery,
                       CASE WHEN UPPER(TRIM(carrier_on_time_overall)) = 'TRUE' THEN 1
                            WHEN UPPER(TRIM(carrier_on_time_overall)) = 'FALSE' THEN 0
                            ELSE NULL
                       END carrier_on_time_overall,                         
                       -- String converted to Date
                       STR_TO_DATE(TRIM(SUBSTRING(pickup_appointment_time, 1, INSTR(pickup_appointment_time, ' ') - 1)), '%m/%d/%Y') pickup_appointment_date,
                       STR_TO_DATE(TRIM(SUBSTRING(pickup_appointment_time, INSTR(pickup_appointment_time, ' ') + 1)), '%H:%i') pickup_appointment_time,
                       STR_TO_DATE(TRIM(SUBSTRING(delivery_appointment_time, 1, INSTR(delivery_appointment_time, ' ') - 1)), '%m/%d/%Y') delivery_appointment_date,
                       STR_TO_DATE(TRIM(SUBSTRING(delivery_appointment_time, INSTR(delivery_appointment_time, ' ') + 1)), '%H:%i') delivery_appointment_time,
                       -- 'TRUE'/'FALSE' converted to number
                       CASE WHEN UPPER(TRIM(has_mobile_app_tracking)) = 'TRUE' THEN 1
                            WHEN UPPER(TRIM(has_mobile_app_tracking)) = 'FALSE' THEN 0
                            ELSE NULL
                       END has_mobile_app_tracking,                         
                       CASE WHEN UPPER(TRIM(has_macropoint_tracking)) = 'TRUE' THEN 1
                            WHEN UPPER(TRIM(has_macropoint_tracking)) = 'FALSE' THEN 0
                            ELSE NULL
                       END has_macropoint_tracking,                         
                       CASE WHEN UPPER(TRIM(has_edi_tracking)) = 'TRUE' THEN 1
                            WHEN UPPER(TRIM(has_edi_tracking)) = 'FALSE' THEN 0
                            ELSE NULL
                       END has_edi_tracking,              
                       CASE WHEN UPPER(TRIM(contracted_load)) = 'TRUE' THEN 1
                            WHEN UPPER(TRIM(contracted_load)) = 'FALSE' THEN 0
                            ELSE NULL
                       END contracted_load, 
                       CASE WHEN UPPER(TRIM(load_booked_autonomously)) = 'TRUE' THEN 1
                            WHEN UPPER(TRIM(load_booked_autonomously)) = 'FALSE' THEN 0
                            ELSE NULL
                       END load_booked_autonomously, 
                       CASE WHEN UPPER(TRIM(load_sourced_autonomously)) = 'TRUE' THEN 1
                            WHEN UPPER(TRIM(load_sourced_autonomously)) = 'FALSE' THEN 0
                            ELSE NULL
                       END load_sourced_autonomously, 
                       CASE WHEN UPPER(TRIM(load_was_cancelled)) = 'TRUE' THEN 1
                            WHEN UPPER(TRIM(load_was_cancelled)) = 'FALSE' THEN 0
                            ELSE NULL
                       END load_was_cancelled,
                       processing_date
                FROM   ALL_RECORDS   
                -- Remove duplicated records
                WHERE  record_order = 1
               ) V
       ) V2                         
LEFT JOIN DIM_LOCATION L
ON V2.lane_origin_city = L.city AND V2.lane_origin_state = L.state
LEFT JOIN DIM_LOCATION L2
ON V2.lane_destination_city = L2.city AND V2.lane_destination_state = L2.state                  
LEFT JOIN DIM_DATE D
ON V2.quote_date = D.date
LEFT JOIN DIM_TIME T
ON V2.quote_time = T.time
LEFT JOIN DIM_DATE D2
ON V2.book_date = D2.date
LEFT JOIN DIM_TIME T2
ON V2.book_time = T2.time
LEFT JOIN DIM_DATE D3
ON V2.source_date = D3.date
LEFT JOIN DIM_TIME T3
ON V2.source_time = T3.time
LEFT JOIN DIM_DATE D4
ON V2.pickup_date = D4.date
LEFT JOIN DIM_TIME T4
ON V2.pickup_time = T4.time                  
LEFT JOIN DIM_DATE D5
ON V2.delivery_date = D5.date
LEFT JOIN DIM_TIME T5
ON V2.delivery_time = T5.time    
LEFT JOIN DIM_EQUIPMENT E
ON V2.equipment_type = E.equipment_type
LEFT JOIN DIM_SHIPPER S
ON V2.shipper_name = S.shipper_name
LEFT JOIN DIM_CARRIER C
ON V2.carrier_name = C.carrier_name
LEFT JOIN DIM_SOURCING SO
ON V2.sourcing_channel = SO.sourcing_channel  
LEFT JOIN DIM_DATE D6
ON V2.pickup_appointment_date = D6.date
LEFT JOIN DIM_TIME T6
ON V2.pickup_appointment_time = T6.time     
LEFT JOIN DIM_DATE D7
ON V2.delivery_appointment_date = D7.date
LEFT JOIN DIM_TIME T7
ON V2.delivery_appointment_time = T7.time                
LEFT JOIN DIM_TRACKING TR
ON V2.has_mobile_app_tracking = TR.has_mobile_app_tracking AND V2.has_macropoint_tracking = TR.has_macropoint_tracking AND V2.has_edi_tracking = TR.has_edi_tracking                     
LEFT JOIN DIM_LOAD LO
ON V2.contracted_load = LO.contracted_load AND V2.load_booked_autonomously = LO.load_booked_autonomously AND V2.load_sourced_autonomously = LO.load_sourced_autonomously                     
""";

cursor.execute(sql)
db.commit()

# Insert dimension LOCATION

In [8]:
cursor = db.cursor()

sql = """
INSERT INTO DIM_LOCATION
(
city,
state,
processing_date
)
SELECT lane_origin_city, 
       lane_origin_state,
       processing_date
FROM   STG_LS       
WHERE  lane_origin_id IS NULL
UNION
SELECT lane_destination_city, 
       lane_destination_state,
       processing_date
FROM   STG_LS       
WHERE  lane_destination_id IS NULL
"""

cursor.execute(sql)
db.commit()

# Insert dimension DATE

In [9]:
cursor = db.cursor()

sql = """
INSERT INTO DIM_DATE
(
date,
fulldate,
day,
day_suffix,
day_name,
day_name_short,
day_of_week,
day_of_year,
week_of_year,
month,
month_name,
month_name_short,
quarter,
quarter_name,
year,
MMYYYY,
first_day_of_month,
last_day_of_month,
is_weekend,
processing_date
)
SELECT date,
       fulldate,
       day,
       day_suffix,
       day_name,
       day_name_short,
       day_of_week,
       day_of_year,
       week_of_year,
       month,
       month_name,
       month_name_short,
       quarter,
       quarter_name,
       year,
       MMYYYY,
       first_day_of_month,
       last_day_of_month,
       is_weekend,
       processing_date
FROM   (SELECT quote_date date,
               DATE_FORMAT(quote_date,'%d/%m/%Y') fulldate,
               DAY(quote_date) day,
               CASE WHEN DAY(quote_date) = 1 THEN '1st'
                    WHEN DAY(quote_date) = 2 THEN '2nd' 
                    WHEN DAY(quote_date) = 3 THEN '3rd'
                    ELSE CONCAT(DAY(quote_date), 'th') 
               END day_suffix,
               DAYNAME(quote_date) day_name,
               SUBSTRING(DAYNAME(quote_date), 1, 3) day_name_short,
               WEEKDAY(quote_date) day_of_week,
               DAYOFYEAR(quote_date) day_of_year,
               WEEKOFYEAR(quote_date) week_of_year,
               MONTH(quote_date) month,
               MONTHNAME(quote_date) month_name,
               SUBSTRING(MONTHNAME(quote_date), 1, 3) month_name_short,
               QUARTER(quote_date) quarter,
               CASE QUARTER(quote_date) 
                   WHEN 1 THEN 'First' 
                   WHEN 2 THEN 'Second'
                   WHEN 3 THEN 'Third'
                   WHEN 4 THEN 'Fourth'
               END quarter_name,
               YEAR(quote_date) year,
               DATE_FORMAT(quote_date,'%m%Y') MMYYYY,
               CAST(CONCAT(SUBSTRING(EXTRACT(YEAR_MONTH FROM quote_date), 1, 4), SUBSTRING(EXTRACT(YEAR_MONTH FROM quote_date), 5, 2), '01') AS DATE) first_day_of_month,
               LAST_DAY(quote_date) last_day_of_month,
               CASE WHEN WEEKDAY(quote_date) IN (0, 1, 2, 3, 4) THEN 0
                    WHEN WEEKDAY(quote_date) IN (5, 6)          THEN 1
                    ELSE NULL
               END is_weekend,
               processing_date
        FROM   STG_LS       
        WHERE  quote_date_id IS NULL
        UNION
        SELECT book_date date,
               DATE_FORMAT(book_date,'%d/%m/%Y') fulldate,
               DAY(book_date) day,
               CASE WHEN DAY(book_date) = 1 THEN '1st'
                    WHEN DAY(book_date) = 2 THEN '2nd' 
                    WHEN DAY(book_date) = 3 THEN '3rd'
                    ELSE CONCAT(DAY(book_date), 'th') 
               END day_suffix,
               DAYNAME(book_date) day_name,
               SUBSTRING(DAYNAME(book_date), 1, 3) day_name_short,
               WEEKDAY(book_date) day_of_week,
               DAYOFYEAR(book_date) day_of_year,
               WEEKOFYEAR(book_date) week_of_year,
               MONTH(book_date) month,
               MONTHNAME(book_date) month_name,
               SUBSTRING(MONTHNAME(book_date), 1, 3) month_name_short,
               QUARTER(book_date) quarter,
               CASE QUARTER(book_date) 
                   WHEN 1 THEN 'First' 
                   WHEN 2 THEN 'Second'
                   WHEN 3 THEN 'Third'
                   WHEN 4 THEN 'Fourth'
               END quarter_name,
               YEAR(book_date) year,
               DATE_FORMAT(book_date,'%m%Y') MMYYYY,
               CAST(CONCAT(SUBSTRING(EXTRACT(YEAR_MONTH FROM book_date), 1, 4), SUBSTRING(EXTRACT(YEAR_MONTH FROM book_date), 5, 2), '01') AS DATE) first_day_of_month,
               LAST_DAY(book_date) last_day_of_month,
               CASE WHEN WEEKDAY(book_date) IN (0, 1, 2, 3, 4) THEN 0
                    WHEN WEEKDAY(book_date) IN (5, 6)          THEN 1
                    ELSE NULL
               END is_weekend,
               processing_date
        FROM   STG_LS       
        WHERE  book_date_id IS NULL
        UNION
        SELECT source_date date,
               DATE_FORMAT(source_date,'%d/%m/%Y') fulldate,
               DAY(source_date) day,
               CASE WHEN DAY(source_date) = 1 THEN '1st'
                    WHEN DAY(source_date) = 2 THEN '2nd' 
                    WHEN DAY(source_date) = 3 THEN '3rd'
                    ELSE CONCAT(DAY(source_date), 'th') 
               END day_suffix,
               DAYNAME(source_date) day_name,
               SUBSTRING(DAYNAME(source_date), 1, 3) day_name_short,
               WEEKDAY(source_date) day_of_week,
               DAYOFYEAR(source_date) day_of_year,
               WEEKOFYEAR(source_date) week_of_year,
               MONTH(source_date) month,
               MONTHNAME(source_date) month_name,
               SUBSTRING(MONTHNAME(source_date), 1, 3) month_name_short,
               QUARTER(source_date) quarter,
               CASE QUARTER(source_date) 
                   WHEN 1 THEN 'First' 
                   WHEN 2 THEN 'Second'
                   WHEN 3 THEN 'Third'
                   WHEN 4 THEN 'Fourth'
               END quarter_name,
               YEAR(source_date) year,
               DATE_FORMAT(source_date,'%m%Y') MMYYYY,
               CAST(CONCAT(SUBSTRING(EXTRACT(YEAR_MONTH FROM source_date), 1, 4), SUBSTRING(EXTRACT(YEAR_MONTH FROM source_date), 5, 2), '01') AS DATE) first_day_of_month,
               LAST_DAY(source_date) last_day_of_month,
               CASE WHEN WEEKDAY(source_date) IN (0, 1, 2, 3, 4) THEN 0
                    WHEN WEEKDAY(source_date) IN (5, 6)          THEN 1
                    ELSE NULL
               END is_weekend,
               processing_date
        FROM   STG_LS       
        WHERE  source_date_id IS NULL
        UNION
        SELECT pickup_date date,
               DATE_FORMAT(pickup_date,'%d/%m/%Y') fulldate,
               DAY(pickup_date) day,
               CASE WHEN DAY(pickup_date) = 1 THEN '1st'
                    WHEN DAY(pickup_date) = 2 THEN '2nd' 
                    WHEN DAY(pickup_date) = 3 THEN '3rd'
                    ELSE CONCAT(DAY(pickup_date), 'th') 
               END day_suffix,
               DAYNAME(pickup_date) day_name,
               SUBSTRING(DAYNAME(pickup_date), 1, 3) day_name_short,
               WEEKDAY(pickup_date) day_of_week,
               DAYOFYEAR(pickup_date) day_of_year,
               WEEKOFYEAR(pickup_date) week_of_year,
               MONTH(pickup_date) month,
               MONTHNAME(pickup_date) month_name,
               SUBSTRING(MONTHNAME(pickup_date), 1, 3) month_name_short,
               QUARTER(pickup_date) quarter,
               CASE QUARTER(pickup_date) 
                   WHEN 1 THEN 'First' 
                   WHEN 2 THEN 'Second'
                   WHEN 3 THEN 'Third'
                   WHEN 4 THEN 'Fourth'
               END quarter_name,
               YEAR(pickup_date) year,
               DATE_FORMAT(pickup_date,'%m%Y') MMYYYY,
               CAST(CONCAT(SUBSTRING(EXTRACT(YEAR_MONTH FROM pickup_date), 1, 4), SUBSTRING(EXTRACT(YEAR_MONTH FROM pickup_date), 5, 2), '01') AS DATE) first_day_of_month,
               LAST_DAY(pickup_date) last_day_of_month,
               CASE WHEN WEEKDAY(pickup_date) IN (0, 1, 2, 3, 4) THEN 0
                    WHEN WEEKDAY(pickup_date) IN (5, 6)          THEN 1
                    ELSE NULL
               END is_weekend,
               processing_date
        FROM   STG_LS       
        WHERE  pickup_date_id IS NULL
        UNION
        SELECT delivery_date date,
               DATE_FORMAT(delivery_date,'%d/%m/%Y') fulldate,
               DAY(delivery_date) day,
               CASE WHEN DAY(delivery_date) = 1 THEN '1st'
                    WHEN DAY(delivery_date) = 2 THEN '2nd' 
                    WHEN DAY(delivery_date) = 3 THEN '3rd'
                    ELSE CONCAT(DAY(delivery_date), 'th') 
               END day_suffix,
               DAYNAME(delivery_date) day_name,
               SUBSTRING(DAYNAME(delivery_date), 1, 3) day_name_short,
               WEEKDAY(delivery_date) day_of_week,
               DAYOFYEAR(delivery_date) day_of_year,
               WEEKOFYEAR(delivery_date) week_of_year,
               MONTH(delivery_date) month,
               MONTHNAME(delivery_date) month_name,
               SUBSTRING(MONTHNAME(delivery_date), 1, 3) month_name_short,
               QUARTER(delivery_date) quarter,
               CASE QUARTER(delivery_date) 
                   WHEN 1 THEN 'First' 
                   WHEN 2 THEN 'Second'
                   WHEN 3 THEN 'Third'
                   WHEN 4 THEN 'Fourth'
               END quarter_name,
               YEAR(delivery_date) year,
               DATE_FORMAT(delivery_date,'%m%Y') MMYYYY,
               CAST(CONCAT(SUBSTRING(EXTRACT(YEAR_MONTH FROM delivery_date), 1, 4), SUBSTRING(EXTRACT(YEAR_MONTH FROM delivery_date), 5, 2), '01') AS DATE) first_day_of_month,
               LAST_DAY(delivery_date) last_day_of_month,
               CASE WHEN WEEKDAY(delivery_date) IN (0, 1, 2, 3, 4) THEN 0
                    WHEN WEEKDAY(delivery_date) IN (5, 6)          THEN 1
                    ELSE NULL
               END is_weekend,
               processing_date
        FROM   STG_LS 
        UNION
        SELECT pickup_appointment_date date,
               DATE_FORMAT(pickup_appointment_date,'%d/%m/%Y') fulldate,
               DAY(pickup_appointment_date) day,
               CASE WHEN DAY(pickup_appointment_date) = 1 THEN '1st'
                    WHEN DAY(pickup_appointment_date) = 2 THEN '2nd' 
                    WHEN DAY(pickup_appointment_date) = 3 THEN '3rd'
                    ELSE CONCAT(DAY(pickup_appointment_date), 'th') 
               END day_suffix,
               DAYNAME(pickup_appointment_date) day_name,
               SUBSTRING(DAYNAME(pickup_appointment_date), 1, 3) day_name_short,
               WEEKDAY(pickup_appointment_date) day_of_week,
               DAYOFYEAR(pickup_appointment_date) day_of_year,
               WEEKOFYEAR(pickup_appointment_date) week_of_year,
               MONTH(pickup_appointment_date) month,
               MONTHNAME(pickup_appointment_date) month_name,
               SUBSTRING(MONTHNAME(pickup_appointment_date), 1, 3) month_name_short,
               QUARTER(pickup_appointment_date) quarter,
               CASE QUARTER(pickup_appointment_date) 
                   WHEN 1 THEN 'First' 
                   WHEN 2 THEN 'Second'
                   WHEN 3 THEN 'Third'
                   WHEN 4 THEN 'Fourth'
               END quarter_name,
               YEAR(pickup_appointment_date) year,
               DATE_FORMAT(pickup_appointment_date,'%m%Y') MMYYYY,
               CAST(CONCAT(SUBSTRING(EXTRACT(YEAR_MONTH FROM pickup_appointment_date), 1, 4), SUBSTRING(EXTRACT(YEAR_MONTH FROM pickup_appointment_date), 5, 2), '01') AS DATE) first_day_of_month,
               LAST_DAY(pickup_appointment_date) last_day_of_month,
               CASE WHEN WEEKDAY(pickup_appointment_date) IN (0, 1, 2, 3, 4) THEN 0
                    WHEN WEEKDAY(pickup_appointment_date) IN (5, 6)          THEN 1
                    ELSE NULL
               END is_weekend,
               processing_date
        FROM   STG_LS       
        WHERE  pickup_appointment_date_id IS NULL
        UNION
        SELECT delivery_appointment_date date,
               DATE_FORMAT(delivery_appointment_date,'%d/%m/%Y') fulldate,
               DAY(delivery_appointment_date) day,
               CASE WHEN DAY(delivery_appointment_date) = 1 THEN '1st'
                    WHEN DAY(delivery_appointment_date) = 2 THEN '2nd' 
                    WHEN DAY(delivery_appointment_date) = 3 THEN '3rd'
                    ELSE CONCAT(DAY(delivery_appointment_date), 'th') 
               END day_suffix,
               DAYNAME(delivery_appointment_date) day_name,
               SUBSTRING(DAYNAME(delivery_appointment_date), 1, 3) day_name_short,
               WEEKDAY(delivery_appointment_date) day_of_week,
               DAYOFYEAR(delivery_appointment_date) day_of_year,
               WEEKOFYEAR(delivery_appointment_date) week_of_year,
               MONTH(delivery_appointment_date) month,
               MONTHNAME(delivery_appointment_date) month_name,
               SUBSTRING(MONTHNAME(delivery_appointment_date), 1, 3) month_name_short,
               QUARTER(delivery_appointment_date) quarter,
               CASE QUARTER(delivery_appointment_date) 
                   WHEN 1 THEN 'First' 
                   WHEN 2 THEN 'Second'
                   WHEN 3 THEN 'Third'
                   WHEN 4 THEN 'Fourth'
               END quarter_name,
               YEAR(delivery_appointment_date) year,
               DATE_FORMAT(delivery_appointment_date,'%m%Y') MMYYYY,
               CAST(CONCAT(SUBSTRING(EXTRACT(YEAR_MONTH FROM delivery_appointment_date), 1, 4), SUBSTRING(EXTRACT(YEAR_MONTH FROM delivery_appointment_date), 5, 2), '01') AS DATE) first_day_of_month,
               LAST_DAY(delivery_appointment_date) last_day_of_month,
               CASE WHEN WEEKDAY(delivery_appointment_date) IN (0, 1, 2, 3, 4) THEN 0
                    WHEN WEEKDAY(delivery_appointment_date) IN (5, 6)          THEN 1
                    ELSE NULL
               END is_weekend,
               processing_date
        FROM   STG_LS       
        WHERE  delivery_appointment_date_id IS NULL 
       ) V
"""

cursor.execute(sql)
db.commit()

# Insert dimension TIME

In [10]:
cursor = db.cursor()

sql = """
INSERT INTO DIM_TIME
(
time,
hour,
minute,
time_12,
processing_date
)
SELECT quote_time time,
       HOUR(quote_time) hour,
       MINUTE(quote_time) minute,
       DATE_FORMAT(quote_time, '%h:%i%p') time_12,
       processing_date
FROM   STG_LS  
WHERE  quote_time_id IS NULL
UNION
SELECT book_time time,
       HOUR(book_time) hour,
       MINUTE(book_time) minute,
       DATE_FORMAT(book_time, '%h:%i%p') time_12,
       processing_date
FROM   STG_LS  
WHERE  book_time IS NULL
UNION
SELECT source_time time,
       HOUR(source_time) hour,
       MINUTE(source_time) minute,
       DATE_FORMAT(source_time, '%h:%i%p') time_12,
       processing_date
FROM   STG_LS  
WHERE  source_time IS NULL
UNION
SELECT pickup_time time,
       HOUR(pickup_time) hour,
       MINUTE(pickup_time) minute,
       DATE_FORMAT(pickup_time, '%h:%i%p') time_12,
       processing_date
FROM   STG_LS  
WHERE  pickup_time IS NULL
UNION
SELECT delivery_time time,
       HOUR(delivery_time) hour,
       MINUTE(delivery_time) minute,
       DATE_FORMAT(delivery_time, '%h:%i%p') time_12,
       processing_date
FROM   STG_LS  
WHERE  delivery_time IS NULL
UNION
SELECT pickup_appointment_time time,
       HOUR(pickup_appointment_time) hour,
       MINUTE(pickup_appointment_time) minute,
       DATE_FORMAT(pickup_appointment_time, '%h:%i%p') time_12,
       processing_date
FROM   STG_LS  
WHERE  pickup_appointment_time IS NULL
UNION
SELECT delivery_appointment_time time,
       HOUR(delivery_appointment_time) hour,
       MINUTE(delivery_appointment_time) minute,
       DATE_FORMAT(delivery_appointment_time, '%h:%i%p') time_12,
       processing_date
FROM   STG_LS  
WHERE  delivery_appointment_time IS NULL
"""

cursor.execute(sql)
db.commit()

# Insert dimension EQUIPMENT

In [11]:
cursor = db.cursor()

sql = """
INSERT INTO DIM_EQUIPMENT
(
equipment_type,
processing_date
)
SELECT DISTINCT 
       equipment_type,
       processing_date
FROM   STG_LS       
WHERE  equipment_id IS NULL
"""

cursor.execute(sql)
db.commit()

# Insert dimension SHIPPER

In [12]:
cursor = db.cursor()

sql = """
INSERT INTO DIM_SHIPPER
(
shipper_name,
processing_date
)
SELECT DISTINCT
       shipper_name,
       processing_date
FROM   STG_LS       
WHERE  shipper_id IS NULL
"""

cursor.execute(sql)
db.commit()

# Upsert dimension CARRIER

In [13]:
cursor = db.cursor()

sql = """
INSERT INTO DIM_CARRIER
(
carrier_name,
carrier_rating,
vip_carrier,
carrier_dropped_us_count,
processing_date
)
SELECT DISTINCT
       S.carrier_name,
       S.carrier_rating,
       S.vip_carrier,
       S.carrier_dropped_us_count,
       S.processing_date
FROM   STG_LS S      
WHERE  S.carrier_id IS NULL
/*
Carrier attributes may change with time, but carrier_dropped_us_count is supposed to increase with time. Therefore a lower value than the one already stored will not
be taken into account
*/
OR     S.carrier_dropped_us_count >= (SELECT MAX(carrier_dropped_us_count)
                                      FROM   DIM_CARRIER
                                      WHERE  carrier_name = S.carrier_name)                                      
ON DUPLICATE KEY UPDATE DIM_CARRIER.carrier_rating = S.carrier_rating,
                        DIM_CARRIER.vip_carrier = S.vip_carrier,
                        DIM_CARRIER.carrier_dropped_us_count = S.carrier_dropped_us_count
"""

cursor.execute(sql)
db.commit()

# Insert dimension SOURCING

In [14]:
cursor = db.cursor()

sql = """
INSERT INTO DIM_SOURCING
(
sourcing_channel,
processing_date
)
SELECT DISTINCT
       sourcing_channel,
       processing_date
FROM   STG_LS       
WHERE  sourcing_id IS NULL
"""

cursor.execute(sql)
db.commit()

# Insert dimension TRACKING

In [15]:
cursor = db.cursor()

sql = """
INSERT INTO DIM_TRACKING
(
has_mobile_app_tracking, 
has_macropoint_tracking, 
has_edi_tracking,
processing_date
)
SELECT DISTINCT
       has_mobile_app_tracking,
       has_macropoint_tracking,
       has_edi_tracking,
       processing_date
FROM   STG_LS       
WHERE  tracking_id IS NULL
"""

cursor.execute(sql)
db.commit()

# Insert dimension LOAD

In [16]:
cursor = db.cursor()

sql = """
INSERT INTO DIM_LOAD
(
contracted_load,
load_booked_autonomously,
load_sourced_autonomously,
processing_date
)
SELECT DISTINCT
       contracted_load,
       load_booked_autonomously,
       load_sourced_autonomously,
       processing_date
FROM   STG_LS       
WHERE  load_id IS NULL
"""

cursor.execute(sql)
db.commit()

# Upsert FACTS table

In [17]:
cursor = db.cursor()

sql = """
INSERT INTO FACTS
(
loadsmart_id,
lane_origin_id,
lane_destination_id,
quote_date_id,
quote_time_id,
book_date_id,
book_time_id,
source_date_id,
source_time_id,
pickup_date_id,
pickup_time_id,
delivery_date_id,
delivery_time_id,
book_price,
source_price,
pnl,
mileage,
shipper_id,
carrier_id,
equipment_id,
sourcing_id,
tracking_id,
pickup_appointment_date_id,
pickup_appointment_time_id,
delivery_appointment_date_id,
delivery_appointment_time_id,
carrier_on_time_to_pickup,
carrier_on_time_to_delivery,
carrier_on_time_overall,
load_id,
load_was_cancelled,
processing_date
)
SELECT V.loadsmart_id,
       L.location_id lane_origin_id,
       L2.location_id lane_destination_id,
       D.date_id quote_date_id,
       T.time_id quote_time_id,
       D2.date_id book_date_id,
       T2.time_id book_time_id,
       D3.date_id source_date_id,
       T3.time_id source_time_id,
       D4.date_id pickup_date_id,
       T4.time_id pickup_time_id,
       D5.date_id delivery_date_id,
       T5.time_id delivery_time_id,
       V.book_price,
       V.source_price,
       V.pnl,
       V.mileage,
       S.shipper_id,
       C.carrier_id,
       E.equipment_id,
       SO.sourcing_id,
       TR.tracking_id,
       D6.date_id pickup_appointment_date_id,
       T6.time_id pickup_appointment_time_id,
       D7.date_id delivery_appointment_date_id,
       T7.time_id delivery_appointment_time_id,
       V.carrier_on_time_to_pickup,
       V.carrier_on_time_to_delivery,
       V.carrier_on_time_overall,
       LO.load_id,
       V.load_was_cancelled,
       V.processing_date
FROM   STG_LS V    
LEFT JOIN DIM_LOCATION L
ON V.lane_origin_city = L.city AND V.lane_origin_state = L.state
LEFT JOIN DIM_LOCATION L2
ON V.lane_destination_city = L2.city AND V.lane_destination_state = L2.state                  
LEFT JOIN DIM_DATE D
ON V.quote_date = D.date
LEFT JOIN DIM_TIME T
ON V.quote_time = T.time
LEFT JOIN DIM_DATE D2
ON V.book_date = D2.date
LEFT JOIN DIM_TIME T2
ON V.book_time = T2.time
LEFT JOIN DIM_DATE D3
ON V.source_date = D3.date
LEFT JOIN DIM_TIME T3
ON V.source_time = T3.time
LEFT JOIN DIM_DATE D4
ON V.pickup_date = D4.date
LEFT JOIN DIM_TIME T4
ON V.pickup_time = T4.time                  
LEFT JOIN DIM_DATE D5
ON V.delivery_date = D5.date
LEFT JOIN DIM_TIME T5
ON V.delivery_time = T5.time    
LEFT JOIN DIM_EQUIPMENT E
ON V.equipment_type = E.equipment_type
LEFT JOIN DIM_SHIPPER S
ON V.shipper_name = S.shipper_name
LEFT JOIN DIM_CARRIER C
ON V.carrier_name = C.carrier_name
LEFT JOIN DIM_SOURCING SO
ON V.sourcing_channel = SO.sourcing_channel  
LEFT JOIN DIM_DATE D6
ON V.pickup_appointment_date = D6.date
LEFT JOIN DIM_TIME T6
ON V.pickup_appointment_time = T6.time     
LEFT JOIN DIM_DATE D7
ON V.delivery_appointment_date = D7.date
LEFT JOIN DIM_TIME T7
ON V.delivery_appointment_time = T7.time                
LEFT JOIN DIM_TRACKING TR
ON V.has_mobile_app_tracking = TR.has_mobile_app_tracking AND V.has_macropoint_tracking = TR.has_macropoint_tracking AND V.has_edi_tracking = TR.has_edi_tracking                     
LEFT JOIN DIM_LOAD LO
ON V.contracted_load = LO.contracted_load AND V.load_booked_autonomously = LO.load_booked_autonomously AND V.load_sourced_autonomously = LO.load_sourced_autonomously 
ON DUPLICATE KEY UPDATE FACTS.lane_origin_id = L.location_id,
                        FACTS.lane_destination_id = L2.location_id,
                        FACTS.quote_date_id = D.date_id,
                        FACTS.quote_time_id = T.time_id,
                        FACTS.book_date_id = D2.date_id,
                        FACTS.book_time_id = T2.time_id,
                        FACTS.source_date_id = D3.date_id,
                        FACTS.source_time_id = T3.time_id,
                        FACTS.pickup_date_id = D4.date_id,
                        FACTS.pickup_time_id = T4.time_id,
                        FACTS.delivery_date_id = D5.date_id,
                        FACTS.delivery_time_id = T5.time_id,
                        FACTS.book_price = V.book_price,
                        FACTS.source_price = V.source_price,
                        FACTS.pnl = V.pnl,
                        FACTS.mileage = V.mileage,
                        FACTS.shipper_id = S.shipper_id,
                        FACTS.carrier_id = C.carrier_id,
                        FACTS.equipment_id = E.equipment_id,
                        FACTS.sourcing_id = SO.sourcing_id,
                        FACTS.tracking_id = TR.tracking_id,
                        FACTS.pickup_appointment_date_id = D6.date_id,
                        FACTS.pickup_appointment_time_id = T6.time_id,
                        FACTS.delivery_appointment_date_id = D7.date_id,
                        FACTS.delivery_appointment_time_id = T7.time_id,
                        FACTS.carrier_on_time_to_pickup = V.carrier_on_time_to_pickup,
                        FACTS.carrier_on_time_to_delivery = V.carrier_on_time_to_delivery,
                        FACTS.carrier_on_time_overall = V.carrier_on_time_overall,
                        FACTS.load_id = LO.load_id,
                        FACTS.load_was_cancelled = V.load_was_cancelled
"""

cursor.execute(sql)
db.commit()

# Database connection is closed

In [18]:
db.close()